In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import timedelta, date
from time import sleep
import os

In [62]:
# 초기화

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)

driver.get('http://marketdata.krx.co.kr/contents/MKD/04/0406/04060100/MKD04060100.jsp')

def fc(selector):
    driver.find_element_by_css_selector(selector).click()
    sleep(1)
    
fc('body > div.html-wrap > div.header-wrap > div > div.breadcrumb-wrap > div > div:nth-child(3) > button')
fc('body > div.html-wrap > div.header-wrap > div > div.breadcrumb-wrap > div > div:nth-child(3) > ul > li:nth-child(4) > a')
fc('body > div.html-wrap > div.header-wrap > div > div.breadcrumb-wrap > div > div:nth-child(4)')
fc('body > div.html-wrap > div.header-wrap > div > div.breadcrumb-wrap > div > div:nth-child(4) > ul > li:nth-child(2) > a')
fc('#market_gubun1a1d0c6e83f027327d8461063f4ac58a6') # 코스피만 받는다

In [63]:
def find_and_download(datetime_str):
    os.system('rm ~/Downloads/data.csv')
    
    date_form = driver.find_element_by_css_selector('#schdate6c8349cc7260ae62e3b1396831a8398f')
    date_form.send_keys(Keys.CONTROL, 'a')
    date_form.send_keys('\b')
    date_form.send_keys(datetime_str)

    fc('#btnidd645920e395fedad7bbbed0eca3fe2e0')
    sleep(5)

    fc('#d9d4f495e875a2e075a1a4a6e1b9770f > button:nth-child(7)')
    sleep(5)
    
    os.system(f'mv ~/Downloads/data.csv ./data/{datetime_str}.csv')

In [7]:
# 오늘까지 크롤링

d = date(2001, 1, 1)
t = date.today()

while d < t:
    d_str = d.strftime('%Y%m%d')
    find_and_download(d_str)
    d += timedelta(1)

In [64]:
# 누락된 파일 재시도

filelist = os.listdir('./data')
filelist = [x.split('.')[0] for x in filelist]

d = date(2001, 1, 1)
t = date.today()

while d < t:
    d_str = d.strftime('%Y%m%d')
    if d_str not in filelist:
        print(d_str)
        find_and_download(d_str)
    d += timedelta(1)


In [66]:
# 주말 제외한 휴일 재시도
d = date(2001, 1, 1)
t = date.today()

while d < t:
    d_str = d.strftime('%Y%m%d')
    filepath = './data/' + d_str + '.csv'

    # 주말이 아닌데 빈 파일인 경우
    if os.stat(filepath).st_size < 1024 and d.weekday() not in [5, 6]:
        find_and_download(d_str)
        
    d += timedelta(1)

In [68]:
# MD5 중복된 파일 검사 후 재시도

import hashlib

filelist = os.listdir('./data')
filelist.sort()

md5_dic = {}
for filename in filelist:
    filepath = './data/' + filename
    
    if os.stat(filepath).st_size < 1024:  # 1K 이하 파일은 거른다
        continue
    
    with open(filepath, 'rb') as f:
        k = hashlib.md5(f.read()).hexdigest()
    
    if k in md5_dic:
        md5_dic[k].append(filepath)
    else:
        md5_dic[k] = [filepath]

same_files = []
for k in md5_dic:
    if len(md5_dic[k]) > 1:
        print(md5_dic[k])
        same_files += md5_dic[k]
        
retry_dates = [x.split('/')[-1][:8] for x in same_files]

for date_str in retry_dates:
    find_and_download(date_str)

In [2]:
# 빈 파일(<1k) 다 지우기

filelist = os.listdir('./data')
filelist.sort()

for filename in filelist:
    filepath = './data/' + filename
    
    if os.stat(filepath).st_size < 1024:
        print(filepath)
        os.system('rm ' + filepath)


./data/20010101.csv
./data/20010106.csv
./data/20010107.csv
./data/20010113.csv
./data/20010114.csv
./data/20010120.csv
./data/20010121.csv
./data/20010123.csv
./data/20010124.csv
./data/20010125.csv
./data/20010127.csv
./data/20010128.csv
./data/20010203.csv
./data/20010204.csv
./data/20010210.csv
./data/20010211.csv
./data/20010217.csv
./data/20010218.csv
./data/20010224.csv
./data/20010225.csv
./data/20010301.csv
./data/20010303.csv
./data/20010304.csv
./data/20010310.csv
./data/20010311.csv
./data/20010317.csv
./data/20010318.csv
./data/20010324.csv
./data/20010325.csv
./data/20010331.csv
./data/20010401.csv
./data/20010405.csv
./data/20010407.csv
./data/20010408.csv
./data/20010414.csv
./data/20010415.csv
./data/20010421.csv
./data/20010422.csv
./data/20010428.csv
./data/20010429.csv
./data/20010501.csv
./data/20010505.csv
./data/20010506.csv
./data/20010512.csv
./data/20010513.csv
./data/20010519.csv
./data/20010520.csv
./data/20010526.csv
./data/20010527.csv
./data/20010602.csv


./data/20041128.csv
./data/20041204.csv
./data/20041205.csv
./data/20041211.csv
./data/20041212.csv
./data/20041218.csv
./data/20041219.csv
./data/20041225.csv
./data/20041226.csv
./data/20041231.csv
./data/20050101.csv
./data/20050102.csv
./data/20050108.csv
./data/20050109.csv
./data/20050115.csv
./data/20050116.csv
./data/20050122.csv
./data/20050123.csv
./data/20050129.csv
./data/20050130.csv
./data/20050205.csv
./data/20050206.csv
./data/20050208.csv
./data/20050209.csv
./data/20050210.csv
./data/20050212.csv
./data/20050213.csv
./data/20050219.csv
./data/20050220.csv
./data/20050226.csv
./data/20050227.csv
./data/20050301.csv
./data/20050305.csv
./data/20050306.csv
./data/20050312.csv
./data/20050313.csv
./data/20050319.csv
./data/20050320.csv
./data/20050326.csv
./data/20050327.csv
./data/20050402.csv
./data/20050403.csv
./data/20050405.csv
./data/20050409.csv
./data/20050410.csv
./data/20050416.csv
./data/20050417.csv
./data/20050423.csv
./data/20050424.csv
./data/20050430.csv


./data/20090104.csv
./data/20090110.csv
./data/20090111.csv
./data/20090117.csv
./data/20090118.csv
./data/20090124.csv
./data/20090125.csv
./data/20090126.csv
./data/20090127.csv
./data/20090131.csv
./data/20090201.csv
./data/20090207.csv
./data/20090208.csv
./data/20090214.csv
./data/20090215.csv
./data/20090221.csv
./data/20090222.csv
./data/20090228.csv
./data/20090301.csv
./data/20090307.csv
./data/20090308.csv
./data/20090314.csv
./data/20090315.csv
./data/20090321.csv
./data/20090322.csv
./data/20090328.csv
./data/20090329.csv
./data/20090404.csv
./data/20090405.csv
./data/20090411.csv
./data/20090412.csv
./data/20090418.csv
./data/20090419.csv
./data/20090425.csv
./data/20090426.csv
./data/20090501.csv
./data/20090502.csv
./data/20090503.csv
./data/20090505.csv
./data/20090509.csv
./data/20090510.csv
./data/20090516.csv
./data/20090517.csv
./data/20090523.csv
./data/20090524.csv
./data/20090530.csv
./data/20090531.csv
./data/20090606.csv
./data/20090607.csv
./data/20090613.csv


./data/20130728.csv
./data/20130803.csv
./data/20130804.csv
./data/20130810.csv
./data/20130811.csv
./data/20130815.csv
./data/20130817.csv
./data/20130818.csv
./data/20130824.csv
./data/20130825.csv
./data/20130831.csv
./data/20130901.csv
./data/20130907.csv
./data/20130908.csv
./data/20130914.csv
./data/20130915.csv
./data/20130918.csv
./data/20130919.csv
./data/20130920.csv
./data/20130921.csv
./data/20130922.csv
./data/20130928.csv
./data/20130929.csv
./data/20131003.csv
./data/20131005.csv
./data/20131006.csv
./data/20131009.csv
./data/20131012.csv
./data/20131013.csv
./data/20131019.csv
./data/20131020.csv
./data/20131026.csv
./data/20131027.csv
./data/20131102.csv
./data/20131103.csv
./data/20131109.csv
./data/20131110.csv
./data/20131116.csv
./data/20131117.csv
./data/20131123.csv
./data/20131124.csv
./data/20131130.csv
./data/20131201.csv
./data/20131207.csv
./data/20131208.csv
./data/20131214.csv
./data/20131215.csv
./data/20131221.csv
./data/20131222.csv
./data/20131225.csv


./data/20170325.csv
./data/20170326.csv
./data/20170401.csv
./data/20170402.csv
./data/20170408.csv
./data/20170409.csv
./data/20170415.csv
./data/20170416.csv
./data/20170422.csv
./data/20170423.csv
./data/20170429.csv
./data/20170430.csv
./data/20170501.csv
./data/20170503.csv
./data/20170505.csv
./data/20170506.csv
./data/20170507.csv
./data/20170509.csv
./data/20170513.csv
./data/20170514.csv
./data/20170520.csv
./data/20170521.csv
./data/20170527.csv
./data/20170528.csv
./data/20170603.csv
./data/20170604.csv
./data/20170606.csv
./data/20170610.csv
./data/20170611.csv
./data/20170617.csv
./data/20170618.csv
./data/20170624.csv
./data/20170625.csv
./data/20170701.csv
./data/20170702.csv
./data/20170708.csv
./data/20170709.csv
./data/20170715.csv
./data/20170716.csv
./data/20170722.csv
./data/20170723.csv
./data/20170729.csv
./data/20170730.csv
./data/20170805.csv
./data/20170806.csv
./data/20170812.csv
./data/20170813.csv
./data/20170815.csv
./data/20170819.csv
./data/20170820.csv
